# Persistence analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ripser import ripser
from persim import plot_diagrams
import matplotlib
plt.rcParams.update(plt.rcParamsDefault)